In [80]:
# 1 创建数据集
def loadDataSet():
    dataSet=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
        ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
        ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
        ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
        ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop',
        'him'],
        ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']] #切分好的词条

    classVec = [0,1,0,1,0,1] #类别标签向量，1代表侮辱性词汇，0代表非侮辱性词汇
    return dataSet,classVec

In [81]:
dataSet,classVec=loadDataSet()
dataSet,classVec# 数据和分类标签

([['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
  ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
  ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
  ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
  ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
  ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']],
 [0, 1, 0, 1, 0, 1])

In [82]:
# 2 构建词汇表：将切分的样本词条整理成不重复的词汇表
def createVocabList(dataSet):
    vocabSet = set()                         #创建一个空的集合
    for doc in dataSet:                      #遍历dataSet中的每一条言论
        vocabSet = vocabSet | set(doc)       #取并集
        vocabList = list(vocabSet)
    return vocabList

In [83]:
vocabList = createVocabList(dataSet)
print(vocabList)     #返回列表

['cute', 'buying', 'not', 'flea', 'dalmation', 'to', 'stupid', 'love', 'has', 'garbage', 'licks', 'take', 'please', 'park', 'problems', 'ate', 'mr', 'so', 'food', 'dog', 'worthless', 'quit', 'him', 'maybe', 'I', 'how', 'steak', 'my', 'stop', 'posting', 'help', 'is']


In [84]:
# 3 获得训练集向量
# 3.1 生成词向量：词条向量化，根据词汇表将切分好的词条向量化，向量的每个元素为1或0
'''
参数：
    vocabList：词汇表
    inputSet：切分好的词条列表中的一条
返回：
    returnVec：文档向量,词集模型
'''
def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0] * len(vocabList)      # 创建一个其中所含元素都为0的向量
    for word in inputSet:                 # 遍历每个词条
        if word in vocabList:             # 如果词条存在于词汇表中，则变为1
            returnVec[vocabList.index(word)] = 1
        else:
            print(f" {word} is not in my Vocabulary!" )
    return returnVec 

In [85]:
# 3.2生成训练集向量列表：词条向量列表
'''
参数说明：
    dataSet：切分好的样本词条
返回：
    trainMat：所有的词条向量组成的列表
'''
def get_trainMat(dataSet):
    trainMat = []                                     #初始化向量列表
    vocabList = createVocabList(dataSet)              #生成词汇表
    for inputSet in dataSet:                          #遍历样本词条中的每一条样本
        returnVec=setOfWords2Vec(vocabList, inputSet) #将当前词条向量化
        trainMat.append(returnVec)                    #追加到向量列表中
    return trainMat

In [86]:
trainMat = get_trainMat(dataSet)
print(trainMat)

[[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0], [0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [87]:
# 4 朴素贝叶斯分类器训练函数
'''
参数:
    trainMat: 训练文档矩阵
    classVec: 训练类别标签向量（分类好的标签）
返回：
    p0V: 非侮辱类的条件概率数组
    p1V: 侮辱类的条件概率数组
    pAb: 文档属于侮辱类的概率
'''
import numpy as np 
def trainNB(trainMat,classVec):
    n = len(trainMat)                     #计算训练的文档数目
    m = len(trainMat[0])                  #计算每篇文档的词条数
    pAb = sum(classVec)/n                 #文档属于侮辱类的概率
    p0Num = np.ones(m)                    #词条出现数初始化为1，避免后续计算分类概率的时候出现0，影响判断
    p1Num = np.ones(m)                    #词条出现数初始化为1，避免后续计算分类概率的时候出现0，影响判断
    p0Denom = 2                           #分母初始化为2，避免后续计算分类概率的时候出现0，影响判断
    p1Denom = 2                           #分母初始化为2，避免后续计算分类概率的时候出现0，影响判断
    for i in range(n):                    #遍历每一个文档
        if classVec[i] == 1:              #统计属于侮辱类的条件概率所需的数据
            p1Num += trainMat[i]
            p1Denom += sum(trainMat[i])
        else:                             #统计属于非侮辱类的条件概率所需的数据
            p0Num += trainMat[i]
            p0Denom += sum(trainMat[i])
    p1V = np.log(p1Num/p1Denom)
    p0V = np.log(p0Num/p0Denom)
    return p0V,p1V,pAb                    #返回属于非侮辱类,侮辱类和文档属于侮辱类的概率

In [88]:
p0V,p1V,pAb = trainNB(trainMat,classVec)
p0V,p1V,pAb

(array([-2.56494936, -3.25809654, -3.25809654, -2.56494936, -2.56494936,
        -2.56494936, -3.25809654, -2.56494936, -2.56494936, -3.25809654,
        -2.56494936, -3.25809654, -2.56494936, -3.25809654, -2.56494936,
        -2.56494936, -2.56494936, -2.56494936, -3.25809654, -2.56494936,
        -3.25809654, -3.25809654, -2.15948425, -3.25809654, -2.56494936,
        -2.56494936, -2.56494936, -1.87180218, -2.56494936, -3.25809654,
        -2.56494936, -2.56494936]),
 array([-3.04452244, -2.35137526, -2.35137526, -3.04452244, -3.04452244,
        -2.35137526, -1.65822808, -3.04452244, -3.04452244, -2.35137526,
        -3.04452244, -2.35137526, -3.04452244, -2.35137526, -3.04452244,
        -3.04452244, -3.04452244, -3.04452244, -2.35137526, -1.94591015,
        -1.94591015, -2.35137526, -2.35137526, -2.35137526, -3.04452244,
        -3.04452244, -3.04452244, -3.04452244, -2.35137526, -2.35137526,
        -3.04452244, -3.04452244]),
 0.5)

In [89]:
# 5 测试朴素贝叶斯分类器
'''
参数说明：
    vec2Classify：待分类的词条数组
    p0V：非侮辱类的条件概率数组
    p1V：侮辱类的条件概率数组
    pAb：文档属于侮辱类的概率
返回：
    0：属于非侮辱类
    1：属于侮辱类
'''
from functools import reduce 
def classifyNB(vec2Classify, p0V, p1V, pAb):
    p1 = sum(vec2Classify * p1V) + np.log(pAb)    #对应元素相乘
    p0 = sum(vec2Classify * p0V) + np.log(1- pAb) #对应元素相乘
    print('p0:',p0)
    print('p1:',p1)
    if p1 > p0:
        return 1
    else:
        return 0

In [90]:
# 测试样本 
def testingNB(testVec):
    dataSet,classVec = loadDataSet()             #创建实验数据
    vocabList = createVocabList(dataSet)         #创建词汇表
    trainMat = get_trainMat(dataSet)             #生成词条向量列表
    p0V,p1V,pAb = trainNB(trainMat,classVec)     #训练朴素贝叶斯分类器，得到分类结果
    thisone = setOfWords2Vec(vocabList, testVec) #生成词向量
    if classifyNB(thisone,p0V,p1V,pAb) == 1:     #判断分类结果
        print(testVec,'属于侮辱类')              #执行分类并打印分类结果
    else:
        print(testVec,'属于非侮辱类')

In [91]:
#测试样本1
testVec1 = ['love','my', 'dalmation']
testingNB(testVec1)

p0: -7.694848072384611
p1: -9.826714493730215
['love', 'my', 'dalmation'] 属于非侮辱类


In [92]:
#测试样本2
testVec2 = ['stupid', 'garbage']
testingNB(testVec2)

p0: -7.20934025660291
p1: -4.702750514326955
['stupid', 'garbage'] 属于侮辱类
